# KD Tree for Bottleneck TSP

In [ ]:
import random
import multiprocessing
from functools import partial

import pandas as pd
import numpy as np
import scipy.spatial
from scipy.spatial import cKDTree
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
sns.set_theme()

from typing import List, Tuple

## Load data

### Option 1: Generate points

In [ ]:
NUM_POINTS = 500
NUM_FEATURES = 12

points = np.random.rand(NUM_POINTS, NUM_FEATURES)

### Option 2: Load points from CSV

In [ ]:
# df = pd.read_csv("features.csv")
df = pd.read_csv("features_no_mode.csv")
points = np.array(df)

## Compute the best path through the points

In [ ]:
class KDTreeBottleneckTSP:
    def __init__(self, *, points: np.ndarray):
        self.points = points
        self.num_points = self.points.shape[0]

    def build_tree(self) -> cKDTree:
        tree = cKDTree(self.points)
        return tree

    def greedy_nearest_path(self, tree: cKDTree, source: int) -> Tuple[List[int], float]:
        """
        Starting from the source node, construct a list of node indices by
        iteratively adding the nearest neighbor in the KD Tree.
        """
        current = source  # The node we're currently at
        visited = {source}  # Set of visited indices
        path = [source]  # The path so far (in order)
        max_dist = 0  # Length of the longest edge in the path

        # Loop until we've visited every point
        for i in range(self.num_points):
            # Start with the closest neighbor, keep incrementing until we find an unvisited neighbor
            for k in range(2, self.num_points):
                # Try to get the closes unvisited neighbor to the current node
                (distance,), (candidate_point,) = tree.query(self.points[current], k=[k])
                if candidate_point not in visited:  # Once an unvisited nearest neighbor is found
                    current = candidate_point  # Move to the neighbor node
                    visited.add(candidate_point)  # Mark the neighbor node as visited
                    path.append(candidate_point)  # Add the neighbor node to the path
                    found_unvisited_neighbor = True
                    if distance > max_dist:
                        max_dist = distance  # Update the maximum edge length in the page
                    break
        return path, max_dist

    def get_best_path(self):
        tree = self.build_tree()

        with multiprocessing.Pool(multiprocessing.cpu_count() - 2) as pool:
            tasks = pool.imap(
                partial(self.greedy_nearest_path, tree),
                range(self.num_points),
                chunksize=4
            )
            paths, max_dists = zip(*(task for task in tqdm(tasks, total=self.num_points)))
            best_path_idx = np.argmin(max_dists)
            best_path = paths[best_path_idx]
            min_max_dist = max_dists[best_path_idx]
            return dict(
                paths=paths,
                max_dists=max_dists,
                best_path_idx=best_path_idx,
                best_path=best_path,
                min_max_dist=min_max_dist,
            )

In [ ]:
optimizer = KDTreeBottleneckTSP(points=points)
results = optimizer.get_best_path()
best_path = results["best_path"]
max_dist = results["min_max_dist"]
print(f"Max edge length: {max_dist}")
print(best_path)

In [ ]:
sns.lineplot(data=(results["max_dists"]))